<h1 align="center"><font size="5">Battle of Neighbourhoods</font></h1>

#### This notebook is used to implement the Coursera Capstone project 

<h2>Table of content</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li><a href="#Q1">Question 1</a></li>
    <li><a href="#Q2">Question 2</a></li>
    <li><a href="#Q3">Question 3</a></li>
</ol>
</div>
<hr>

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print('Libraries imported.')

Libraries imported.


# I. Question 1 <a class="anchor" id="Q1"></a>

#### Since it's a simple scraping, we'll only use *Pandas* to do so

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #Link to the page
tables = pd.read_html(url) #Read any existing tables in the link 
data = tables[0] #Select the wanted table
data.columns = ["PostalCode", "Borough", "Neighbourhood"] #Name the columns as indicated
data.head() #Show the firs five rows

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Now, we should clean our table, as indicated, every *Not assigned* "Borough" row will be dropped. To do so, we should first replace it with *Nan* so python could recognize it and drop the entire row

In [3]:
data["Borough"].replace("Not assigned",np.nan, inplace = True) #Replace "Not assigned" with "Nan"

In [4]:
data.dropna(inplace = True) #Drop rows with "Not assigned" Borough

In [5]:
data.head() #Show the first five rows

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### The second thing to check is if there's any *Not assigned* "Neighbourhood" cells. If yes, we'll assign to it the same as the Borough cell

In [6]:
exists = "Not assigned" in data.Neighbourhood #Check if there's any "Not assigned" Neighborhouud cells
exists

False

#### We notice that there's no *Not assigned* "Neighbourhood" cell

#### Finally, let's check the DataFrame size

In [7]:
data.shape

(103, 3)

# II. Question 2 <a class="anchor" id="Q2"></a>

#### Since the *geocoder* didn't work for me, I'll proceed with the *csv* file containing the latitude and longitude informations

In [8]:
long_lat=pd.read_csv(r"https://cocl.us/Geospatial_data") #Read the csv file
long_lat=pd.DataFrame(long_lat) #Turn it to a Pandas DataFrame
long_lat.head() #Show the first five rows

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Before assembling the two tables in one final table, let's sort the *data* table so it becomes in the same order as the *long_lat* table

In [9]:
data_sorted = data.sort_values("PostalCode") #Sort the "long_lat" table in the ascending order
data_sorted.shape

(103, 3)

#### Now, let's drop the "Postal Code" column from the *long_lat* table and assemble the two tables in one final table

In [10]:
long_lat.drop(["Postal Code"], axis = 1, inplace = True) #Drop the "Postal Code" column from the "long_lat" table
long_lat.head() #Show the first five rows

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [11]:
data_sorted.reset_index(drop=True, inplace=True) #Reset index
long_lat.reset_index(drop=True, inplace=True) #Reset index
final_data = pd.concat([data_sorted, long_lat], axis = 1) #Concatenate the two dataframes
final_data.head() #Show the forst five five rows of the final dataframe

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Let's check if the concatenation is done correctly

In [12]:
exists = "NaN" in data_sorted #Check if there's any "Nan" cells after the concatenation to make sure it's done correctly
exists

False

# III. Question 3 <a class="anchor" id="Q3"></a>

#### First, let's get the geographical coordinates of *Toronto City* and visualize it using *Folium* maps

#### Try zooming on and clicking the blue circles in the map, informations about the borough, the neighbourhooud and the postal code will appear

In [13]:
#Get the coordinates of Toronto City
address = 'Toronto City'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto City are {}, {}.'.format(latitude, longitude))
#Plot the map using Folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#Add markers to the map
for lat, long, borough, neigh, pstlcd in zip(final_data['Latitude'], final_data['Longitude'], final_data['Borough'], final_data['Neighbourhood'], final_data['PostalCode']):
    label = '{}; {}; {}'.format(neigh, borough, pstlcd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, long], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)     
map_toronto

The geographical coordinates of Toronto City are 43.6534817, -79.3839347.


#### Let's simplify the task and focus on clustering and segmenting only *Toronto* neighbourhoods

#### To do so, whe have to slice the previous DataFrame to a new one containing only *Toronto* neighbourhoods

In [14]:
toronto_data = final_data[final_data['Borough'].str.contains('Toronto')].reset_index(drop=True) #New DataFrame containg only Toronto neighbourhoods
toronto_data.head() #Show the first five rows

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Let's visualize these neighbourhoods using *Folium*

In [15]:
#Plot the new map using Folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#Add markers to the map
for lat, long, borough, neigh, pstlcd in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood'], toronto_data['PostalCode']):
    label = '{}; {}; {}'.format(neigh, borough, pstlcd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, long], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)     
map_toronto

#### Next, we are going to use the Foursquare API to explore the neighbourhoods and segment them

#### Let's define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'RLHUH1EHWMKBYQZQ5FOQU1JEJZSHUQYATCPNPC2GPEU1D3ZF' #Foursquare ID
CLIENT_SECRET = 'DUENGJFSMN2DAVWX0J3T1UDTDOIKLBZQRBLX4G5JTWYENNSS' #Foursquare Secret
VERSION = '20180605' #Foursquare API version
LIMIT = 100 #Foursquare API limit value
radius = 500 #Define radius
#Create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT)

#### The next function will get the *top 100 venues*, if available, that are in the neighbourhoods within a *500 meters* radius and store all the informations in a DataFrame

In [17]:
#Function that extracts the category of the venue
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)      
        #Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)     
        #Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #Return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    #Put everything into a Pandas DataFrame
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'] 
    return(nearby_venues)

#### Let's test the function and get the DataFrame containaing all the venues categories in Toronto

In [18]:
#Test the  function to verify its output
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude'])
toronto_venues.head() #Show the first five rows

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
Runny

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


#### Next, we apply *One Hot Encoding* on the "Venue Category" column and store it in a new DataFrame, we also bring the "Neighbourhood" column to the new DataFrame and place it first

In [19]:
#One Hot encoding
toronto_dummy= pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#Add neighbourhood column back to dataframe
toronto_dummy['Neighbourhood'] = toronto_venues['Neighbourhood'] 
#Move neighbourhood column to the first column
fixed_columns = [toronto_dummy.columns[-1]] + list(toronto_dummy.columns[:-1])
toronto_dummy = toronto_dummy[fixed_columns]
toronto_dummy.head() #Show the first five rows

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### We can have a look on all the venues categories that exists in our DataFrame

In [20]:
toronto_venues["Venue Category"].unique() #All venues categories

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Fruit & Vegetable Store',
       'Brewery', 'Restaurant', 'Juice Bar', 'Pizza Place', 'Bookstore',
       'Bubble Tea Shop', 'Furniture / Home Store', 'Dessert Shop', 'Spa',
       'Grocery Store', 'Coffee Shop', 'Tibetan Restaurant', 'Bakery',
       'Indian Restaurant', 'Caribbean Restaurant', 'Café',
       'Japanese Restaurant', 'Lounge', 'Frozen Yogurt Shop',
       'Liquor Store', 'American Restaurant', 'Fast Food Restaurant',
       'Gym', 'Fish & Chips Shop', 'Sushi Restaurant', 'Park',
       'Pet Store', 'Steakhouse', 'Movie Theater', 'Sandwich Place',
       'Board Shop', 'Food & Drink Shop', 'Seafood Restaurant', 'Gay Bar',
       'Fish Market', 'Cheese Shop', 'Comfort Food Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store', 'Thai Restaurant',
       'Coworking Space', 'Gastropub', 'Clothing 

#### Now, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [21]:
toronto_grouped = toronto_dummy.groupby('Neighbourhood').mean().reset_index() #Get the mean of frequency of occurence
toronto_grouped.head() #Show the first five rows

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.0000,0.000,0.0,0.0,0.0,0.016949,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.016949,0.050847,0.000000,0.000000,0.0,0.016949,0.016949,0.0,0.033898,0.0,0.0,0.016949,0.0,0.0000,0.0,0.0000,0.0,0.016949,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.016949,0.0,0.0,0.0,0.033898,0.0,0.0,0.0,0.000000,0.016949,0.050847,0.084746,0.0,0.0,0.0,0.0,0.0,0.0,0.016949,0.000000,0.016949,0.000000,0.0,0.0,0.016949,0.0,0.0,0.0,0.0,0.016949,0.0,0.016949,0.000000,0.0,0.0,0.0,0.000000,0.0,0.016949,0.0,0.0,0.0,0.0,0.000000,0.033898,0.000000,0.0,0.0,0.016949,0.0,0.0,0.0,0.0,0.0,0.0,0.01694

#### Now, let's create a new DataFrame that contains each neighbourhood along with the *Top 15* avenues in it in the descending order
#### To do so, let's first create a function that will sort the venues in descending order

In [22]:
#Function for descendent order sorting
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Next, we create the DataFrame

In [23]:
num_top_venues = 15
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Beer Bar,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Breakfast Spot,Clothing Store,Sandwich Place,Jazz Club,Bistro,Shopping Mall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Burrito Place,Convenience Store,Restaurant,Italian Restaurant,Stadium,Intersection,Bar,Climbing Gym,Furniture / Home Store,Performing Arts Venue,Grocery Store
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Light Rail Station,Farmers Market,Fast Food Restaurant,Smoke Shop,Comic Shop,Gym / Fitness Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique,Airport Lounge,Airport,Airport Food Court,Gift Shop,Dog Run
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Salad Place,Ramen Restaurant,Portuguese Restaurant,Poke Place,Middle Eastern Restaurant,Park,Office


#### After preparing the DataFrame, it's time to run *Clustering* algorithms on it

#### In my case, I'll proceed wit *K-Means* algorithm based on *4 clusters*

In [24]:
#Number of clusters
kclusters = 4
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', axis = 1)
#Run K-Means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

#### Let's create now, a final DataFrame that contains the previous informations along with the cluster labels and the Top 15 venues

In [25]:
#Add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
toronto_merged = toronto_data
#Merge toronto_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant,Dumpling Restaurant,Diner,Electronics Store,Escape Room,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Spa,Japanese Restaurant,Cosmetics Shop,Juice Bar,Brewery,Café,Restaurant,Liquor Store
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Pub,Sandwich Place,Liquor Store,Board Shop,Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse,Food & Drink Shop,Ice Cream Shop,Brewery,Sushi Restaurant,Pet Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Brewery,Café,Gastropub,Bakery,American Restaurant,Yoga Studio,Coworking Space,Cheese Shop,Clothing Store,Pet Store,Comfort Food Restaurant,Convenience Store,Diner,Neighborhood
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Finally, let's visualize the resulting clusters on the Toronto City map using *Folium*

In [26]:
#Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
#Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)  
map_clusters